In [ ]:
# import modules
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
import pandas as pd
import scipy as sp
from scipy import signal
from scipy import stats
import scipy.signal as scisig
import matplotlib 
from scipy.integrate import simps
matplotlib.rcParams.update({'figure.max_open_warning': 0})
import scikit_posthocs as spost
%matplotlib notebook

In [ ]:
### definitions
peak_prom = 20
smoothed = 'yes'
threshold = 30
frame_rate = 1/10

In [ ]:
def import_data_xana_spinning(excel_name,filename_ending,stimulation_times):
    data_longform = pd.DataFrame(columns = ["Filename","Time","Muscle","value"])
    for i in range(len(stimulation_times)):
        #print(i)
        filename = 'Rawdata_Fiji_Fly'+str(i+1)+filename_ending
        #print(filename)
        dataframe = pd.read_excel(excel_name,sheet_name=filename,skiprows=0)
        #print(dataframe)
        levator_act, depressor_act = fluorescence_measure(dataframe)
        timepoints = dataframe.Frame-stimulation_times[i]
        #print(timepoints)
        dataz = {'Time':timepoints,'Levator':levator_act,'Depressor':depressor_act}
        dataz = pd.DataFrame(dataz)
        dataz["Filename"] = filename
        #print(dataz)
        data_melted = pd.melt(dataz,id_vars=["Filename","Time"],var_name="Muscle")
        #print(data_melted)
        data_longform = data_longform.append(data_melted)
    return data_longform

def fluorescence_measure(dataframe):
    
    #Define Levator and Depressor mean fluorescences
    levator_fluor = dataframe['Mean1']
    depressor_fluor = dataframe['Mean2']
    frames = dataframe['Frame']

    #Find Max and min of fluorescence for both muscles
    levator_fmax = max(levator_fluor)
    levator_fmin = min(levator_fluor)

    depressor_fmax = max(depressor_fluor)
    depressor_fmin = min(depressor_fluor)

    #Normalizing the fluorescence 
    #Mean of each frame minus min
    levator_fnorm = levator_fluor - levator_fmin
    #Maximum minus minimum will give us the highest activity normalized
    levator_maxnorm = levator_fmax - levator_fmin

    #same for depressor muscle
    depressor_fnorm = depressor_fluor - depressor_fmin
    depressor_maxnorm = depressor_fmax - depressor_fmin

    #Calculate percentage of muscles activity
    levator_act = (levator_fnorm*100)/levator_maxnorm
    depressor_act = (depressor_fnorm*100)/depressor_maxnorm
    
    return levator_act, depressor_act

#fluorescence_measure(dataframe)

def scorrelation_muscle_activity(levator_act,depressor_act,times): #looks at the correlation between muscle activity for only the times after the stimulation
    
    #select only the timepoints after the stimulation 
    levator_act = np.array(levator_act)[times >=0]
    depressor_act = np.array(depressor_act)[times >= 0]
    
    #First we need to exclude areas where both muscle activity is null
    levator_less5 = np.where(levator_act<20) #Find where levator activity is smaller than 20%
    depressor_less5 = np.where (depressor_act <20) #Find where depressor activity is smaller than 20%
    timewindow = np.intersect1d(levator_less5, depressor_less5) #Find where both muscles have activity <20%
    #use only the selected frames for the correlation analysis
    #delete from the data the values in which activity of is below 20 for both of the muscles
    levator_select = np.delete(levator_act, timewindow)
    depressor_select= np.delete(depressor_act,timewindow)
    
    #fig,ax = plt.subplots(figsize = [5,5])
    #plt.scatter(levator_select,depressor_select)
    #print(levator_select)
    #print(depressor_select)
    
    if len(levator_select) <2 or len(depressor_select) <2:
        r=np.nan
        pvalue =np.nan
    else:
        r,pvalue = stats.spearmanr(levator_select, depressor_select)
    
    return r,pvalue

def wilcoxonz(data):
    wilcoxon_pval = []
    wilcoxon_stat = []
    for f in list(df):
        #print(f)
        test = data[f]
        #print(test)
        stat, pval = stats.wilcoxon(test) #do a wilcoxon test on the spearmans correlation scores
        wilcoxon_pval.append(pval)
        wilcoxon_stat.append(stat)
    return wilcoxon_stat,wilcoxon_pval

# Import data

In [ ]:
ATR_wstim = import_data_xana_spinning('Fluorescence measures_ATR.xlsx','_Stim',[182,99,155,56,131,50,126,22,153,68,26,27,38])
ATR_nostim = import_data_xana_spinning('Fluorescence measures_ATR.xlsx','_No_Stim',[0,0,0,0,0,0,0,0,0,0,0,0,0])
ATR_decap = import_data_xana_spinning('Fluorescence measures_ATR.xlsx','_Decap',[210,106,60,39,34,239,35,144,214,25,132,68,23])
NoATR_wstim = import_data_xana_spinning('Fluorescence measures_No ATR.xlsx','_Stim',[38,136,13,10,22,77,124,61,129,171,21])
NoATR_decap = import_data_xana_spinning('Fluorescence measures_No ATR.xlsx','_Decap',[69,42,95,87,167,208,271,233,22,80,99,32])


In [ ]:
ATR_wstim["Condition"] = "ATR_wstim"
ATR_nostim["Condition"] = "ATR_nostim"
NoATR_wstim["Condition"] = "NoATR_wstim"
ATR_decap["Condition"] = "ATR_Decap"
NoATR_decap["Condition"] = "NoATR_Decap"

all_conditions = ATR_wstim.append(ATR_nostim)
all_conditions = all_conditions.append(NoATR_wstim)
all_conditions = all_conditions.append(ATR_decap)
all_conditions = all_conditions.append(NoATR_decap)

print(all_conditions)

## Correlation between levator and depressor muscles

In [ ]:
def group_spearmans_corr(dataz):
    all_r = []
    all_pval = []
    for file in dataz.Filename.unique():
        data = dataz.query('Filename == @file')
        scorr = scorrelation_muscle_activity(data.query('Muscle == "Levator"').value,data.query('Muscle == "Depressor"').value,data.query('Muscle == "Levator"').Time)
        #print("spearmans = ",str(scorr))
        #fig, ax = plt.subplots() 
        
        #plt.scatter(data.query('Muscle == "Levator" and Time>0').value,data.query('Muscle == "Depressor" and Time>0').value)
        
        all_r.append(scorr[0])
        all_pval.append(scorr[1])
    return all_r,all_pval

In [ ]:
spear_ATR_stim_r,spear_ATR_stim_pval = group_spearmans_corr(ATR_wstim)

spear_ATR_nostim_r,spear_ATR_nostim_pval = group_spearmans_corr(ATR_nostim)

spear_ATR_decap_r,spear_ATR_decap_pval = group_spearmans_corr(ATR_decap)

spear_noATR_stim_r,spear_noATR_stim_pval = group_spearmans_corr(NoATR_wstim)

spear_noATR_decap_r,spear_noATR_decap_pval = group_spearmans_corr(NoATR_decap)

In [ ]:
#Transform numpy into pandas since goups have different dimensions
df_ATR_Stim = pd.DataFrame(spear_ATR_stim_r, columns = ['ATR Stim'])
df_ATR_NoStim = pd.DataFrame(spear_ATR_nostim_r, columns = ['ATR NoStim'])
df_ATR_Decap = pd.DataFrame(spear_ATR_decap_r,columns = ['ATR Decap'])
df_NoATR_Stim = pd.DataFrame(spear_noATR_stim_r, columns = ['NoATR Stim'])
df_NoATR_Decap = pd.DataFrame(spear_noATR_decap_r, columns = ['NoATR Decap'])


#Concatenate all groups into a dataframe with index label
df = pd.concat([df_NoATR_Stim, df_ATR_NoStim,df_ATR_Stim,df_NoATR_Decap,df_ATR_Decap ],axis=1)
wilk = wilcoxonz(df)

fig, ax = plt.subplots()
ax =sns.boxplot(data=df,palette="GnBu_d")
ax =sns.swarmplot(data=df,color='black')
ax.set(ylim=(-1, 1.1))
sns.despine(offset=10, trim=True)
ax.axhline(0, color = 'grey', ls='--')

plt.show()

plt.savefig('Person Correlation Plot.svg')
plt.savefig('Pearson correlation plot.png')


df_melted = pd.melt(df,var_name = ["Condition"])

post_dunn = spost.posthoc_dunn(df_melted,group_col="Condition",val_col = "value")
print("Post dunn =")
print(post_dunn)
print("")

# Intensity of muscle contraction

In [ ]:
#binz = 15
#
#histog_ATR = np.histogram(ATR_wstim.query('Time >= 0').value,bins = binz)
#histog_ATR_decap = np.histogram(ATR_decap.query('Time >= 0').value,bins = binz)
#
##normalised values together
#fig, ax = plt.subplots(figsize = [7,4])
#ax.bar(histog_ATR[1][:-1],histog_ATR[0]/np.sum(histog_ATR[0]),width = 4,label="ATR with stim")
#ax.bar(histog_ATR_decap[1][:-1]+1,histog_ATR_decap[0]/np.sum(histog_ATR_decap[0]),width = 4,label="ATR decapitated")
#ax.legend()
#
#
#viol_data = all_conditions.query('Time > 0')
#
#fig, ax = plt.subplots(figsize = [4,5])
#ax =sns.boxplot(x = "Condition",y="value",data=viol_data.query('Condition == "ATR with stim" or Condition == "ATR Decap"'),palette="GnBu_d")
##ax =sns.swarmplot(data=df,color='black')
##ax.set(ylim=(-1, 1.1))
#sns.despine(offset=10, trim=True)
##ax.axhline(0, color = 'grey', ls='--')
#plt.show()
#
#fig, ax = plt.subplots(figsize = [4,5])
#ax =sns.violinplot(x = "Condition",y="value",data=viol_data.query('Condition == "ATR with stim" or Condition == "ATR Decap"'),palette=["#539EBA","#3E5660"],cut = 0)
##ax =sns.swarmplot(data=df,color='black')
##ax.set(ylim=(-1, 1.1))
#sns.despine(offset=10, trim=True)
##ax.axhline(0, color = 'grey', ls='--')
#plt.show()
#
#stat,p_val = stats.mannwhitneyu(viol_data.query('Condition == "ATR with stim"').value,viol_data.query('Condition == "ATR Decap"').value,alternative = 'two-sided')
#print('Mann Whitney U stat = ', str(stat),' p value =',str(p_val))

## Number and size of bouts of muscle activity

### Integrations of activity

In [ ]:
def peaks_troughs(data,times,peak_prom = peak_prom,trough_ord = 20,smoothed = 'yes',graph = 'yes',title = "",threshold = threshold):
    times = np.array(times)
    
    peaks = scisig.find_peaks(data,height = 0, prominence = peak_prom)
    peaks_locs = peaks[0]
    peaks_heights = peaks[1]["peak_heights"]
    if threshold > 0:
        locations = peaks_heights > threshold
        peaks_locs = peaks_locs[locations]
        peaks_heights = peaks_heights[locations]
    
    troughs = scisig.argrelmin(data.values,order = trough_ord)
    scatterz2_height = [data.values[x] for x in troughs[0]]
    scatterz = [times[x] for x in peaks_locs]
    scatterz2 = [times[x] for x in troughs[0]]
    
    if smoothed  == 'yes': #can smooth the data with different amounts
        dataz = data
        dataz = dataz.rolling(2,center = True).mean()
        peaks_smooth = scisig.find_peaks(dataz,height = 0, prominence = peak_prom)
        peaks_locs_smooth = peaks_smooth[0]
        peaks_heights_smooth = peaks_smooth[1]["peak_heights"]
        
        if threshold > 0:
            locations_smooth = peaks_heights_smooth > threshold
            peaks_locs_smooth = peaks_locs_smooth[locations_smooth]
            peaks_heights_smooth = peaks_heights_smooth[locations_smooth]
        
        troughs_smooth = scisig.argrelmin(dataz.values,order = trough_ord)
        scatterz2_height_smooth = [dataz.values[x] for x in troughs_smooth[0]]
        scatterz_smooth = [times[x] for x in peaks_locs_smooth]
        scatterz2_smooth = [times[x] for x in troughs_smooth[0]]
    
        
    if graph == 'yes':
        fig2,ax2 = plt.subplots(figsize=(10,4))
        plt.plot(times,data,label = "raw",color='k',zorder = 1)
        plt.scatter(scatterz,peaks_heights,color='b',s=100,zorder = 2,alpha = 0.7,label = "Peak raw")
        plt.scatter(scatterz2,scatterz2_height,color = [0.2,0.7,0.7])
        if smoothed == 'yes':
            plt.plot(times,dataz,label = "smoothed",color = [0.5,0,0],zorder = 3)
            plt.scatter(scatterz_smooth,peaks_heights_smooth,color=[1,0.4,0.4],s = 80,zorder = 4,alpha = 0.7,label = "Peak smoothed")
            plt.scatter(scatterz2_smooth,scatterz2_height_smooth,color = [1,0.7,0.7])
        
        #plt.xlim([0,80])
        plt.title(title,fontsize=20)
        plt.xlabel("Time in s")
        plt.ylabel("Normalised fluorescence")
        plt.legend()
        
    if smoothed == 'yes':
        return dataz,peaks_locs_smooth,peaks_heights_smooth
    else:
        return peaks_locs,peaks_heights

In [ ]:
def muscle_analysis(data,thresh,peak_prom=peak_prom,smoothed='yes',graphs = 'no'): #threshold is below which there are no peaks
    integrations_lev = []
    integrations_dep = []
    peakheights = []
    peaklocs = []
    threshold = thresh #threshold below which we don't consider it a muscle contraction
    for g in data.Filename.unique():
        #print(data.query('Muscle == "Levator" and Filename == @g'))
        test_peaks_lev = peaks_troughs(data.query('Muscle == "Levator" and Filename == @g').value,data.query('Muscle == "Levator" and Filename == @g').Time,smoothed = smoothed,peak_prom = peak_prom,threshold = threshold,title = g+" Levator",graph=graphs)
        #print(test_peaks_lev)
        int_lev = simps(np.array(data.query('Muscle == "Levator" and Filename == @g').value), np.array(data.query('Muscle == "Levator" and Filename == @g').Time))
        if test_peaks_lev[1].shape[0] > 0:
            integrations_lev.append(int_lev/test_peaks_lev[1].shape[0])
        else:
            integrations_lev.append(np.nan)
        for h,i in enumerate(test_peaks_lev[2]):
            peakheights.append(i)
            #peaklocs.append(test_peaks_lev[1][h])
            #print(peaklocs)
            
            ## NOT FINISHED; NOT GOOD HERE FOR LOCATIONS
#
        #print("Total area under curve = ",str(int_lev))
        #print("Average area under each peak = ",str(int_lev/test_peaks_lev[1].shape[0]))
        #print("")
#        
        test_peaks_dep = peaks_troughs(data.query('Muscle == "Depressor" and Filename == @g').value,data.query('Muscle == "Depressor" and Filename == @g').Time,smoothed = smoothed,peak_prom = peak_prom,threshold = threshold,title = g+" Depressor",graph=graphs)
        int_dep = simps(np.array(data.query('Muscle == "Depressor" and Filename == @g').value), np.array(data.query('Muscle == "Depressor" and Filename == @g').Time))
        if test_peaks_dep[1].shape[0] > 0:
            integrations_dep.append(int_dep/test_peaks_dep[1].shape[0])
        else:
            integrations_dep.append(np.nan)
        
        #intergrations_dep.append(int_dep/test_peaks_dep[1].shape[0])
        for i in test_peaks_dep[2]:
            peakheights.append(i)
#            peaklocs.append(i)
#        #print("Total area under curve = ",str(int_dep))
#        #print("Average area under each peak = ",str(int_dep/test_peaks_dep[1].shape[0]))
#        #print("")
#
    
    #print(integrations_lev)
    
    #print(peakheights)
    return integrations_lev,integrations_dep,peakheights#,peaklocs

In [ ]:
peak_heights = pd.DataFrame(columns=["Condition","Height"])
peak_integrations = pd.DataFrame(columns=["Condition","Muscle","Area"])
#only for post stimulation conditions
for each in all_conditions.Condition.unique():
    print(each)
    integration_levator,integration_depressor,peakheight = muscle_analysis(all_conditions.query('Condition == @each and Time >= 0'),threshold,graphs = 'no')
    
    integrat_lev = pd.DataFrame(integration_levator,columns = ["Area"])
    integrat_lev["Condition"] = each
    integrat_lev["Muscle"] = "Levator"
    
    integrat_dep = pd.DataFrame(integration_depressor,columns = ["Area"])
    integrat_dep["Condition"] = each
    integrat_dep["Muscle"] = "Depressor"
    
    peak_integrations = pd.concat([peak_integrations,integrat_lev,integrat_dep])
    #print(peak_integrations)
    
    peakheight = pd.DataFrame(peakheight,columns = ["Height"])
    peakheight["Condition"] = each
    peak_heights = pd.concat([peak_heights,peakheight])
print(peak_heights)
print(peak_integrations)
    

In [ ]:
pval = spost.posthoc_dunn(peak_integrations,group_col="Condition",val_col = "Area")

fig, ax = plt.subplots(figsize = [10,6])
ax =sns.violinplot(x = "Condition",y="Area",data=peak_integrations,palette="GnBu_d")#,cut = 0)
ax =sns.swarmplot(x = "Condition",y="Area",data=peak_integrations,color='k',s=5)

plt.ylabel("Normalised fluorescence",fontsize = 15)
plt.savefig('Peak integrations viol.svg')
plt.savefig('Peak integrations viol.png')


print(pval)

### Peak heights

In [ ]:
pval = spost.posthoc_dunn(peak_heights,group_col="Condition",val_col = "Height")

fig, ax = plt.subplots(figsize = [10,6])
ax =sns.violinplot(x = "Condition",y="Height",data=peak_heights,palette="GnBu_d",cut = 0,order = ["NoATR_wstim","ATR_nostim","ATR_wstim","NoATR_Decap","ATR_Decap"])
#ax =sns.swarmplot(x = "Condition",y="Height",data=peak_heights,color='k',s=3,order = ["NoATR_wstim","ATR_nostim","ATR_wstim","NoATR_Decap","ATR_Decap"])

plt.ylabel("Normalised fluorescence",fontsize = 15)
plt.savefig('Peak heights Plot no points.svg')
#plt.savefig('Peak heights Plot.png')

print(pval)

In [ ]:
binz = np.linspace(30,100,12)
fig, ax = plt.subplots(figsize = [10,5])
for each in peak_heights.Condition.unique():
    histog_heights = np.histogram(peak_heights.query('Condition == @each').Height,bins = binz)
    #print(histog_heights)
    plt.plot(histog_heights[1][:-1],histog_heights[0]/np.sum(histog_heights[0]),label=each,linewidth=5)#,palette="GnBu_d")
    plt.bar(histog_heights[1][:-1],histog_heights[0]/np.sum(histog_heights[0]),width=5,alpha=0.2)#,palette="GnBu_d")
    #ax.bar(histog_ATR_decap_peakheights[1][:-1]+1,histog_ATR_decap_peakheights[0]/np.sum(histog_ATR_decap_peakheights[0]),width = wid,label="ATR decapitated",color=["#3E5660"])
    ax.legend()
    plt.xlabel("Normalised peak height",fontsize = 12)
    plt.ylabel("Proportion",fontsize = 12)
plt.savefig('Peak heights histogram all.svg')
plt.savefig('Peak heights histogram all.png')

### Peak width

In [ ]:
cut_off = 20 #number of fluorescence points to go down to define the start of the peak

def consecutive(data, stepsize=1):
    return np.split(data, np.where(np.diff(data) != stepsize)[0]+1)

def muscle_peak_width(data,thresh,peak_prom=peak_prom,smoothed='yes',graphs = 'no',condition = ""): #threshold is below which there are no peaks
    widths_lev = []
    widths_dep = []
    threshold = thresh #threshold below which we don't consider it a muscle contraction
    print("levator")
    fig, ax = plt.subplots(len(data.Filename.unique()),1,figsize = [6,18])
    print("depressor")
    fig2, ax2 = plt.subplots(len(data.Filename.unique()),1,figsize = [6,18])
    
    for h,g in enumerate(data.Filename.unique()):
        test_peaks_lev = peaks_troughs(data.query('Muscle == "Levator" and Filename == @g').value,data.query('Muscle == "Levator" and Filename == @g').Time,smoothed = smoothed,peak_prom = peak_prom,threshold = threshold,title = g+" Levator",graph=graphs)
        #print(test_peaks_lev[0])
        ax[h].plot(np.arange(test_peaks_lev[0].shape[0])*frame_rate,test_peaks_lev[0])
        ax[h].scatter(0,0,s=0)
        for j, peak in enumerate(test_peaks_lev[1]):
            height_thresh = test_peaks_lev[2][j]-cut_off
            thresh_crossings = np.where(test_peaks_lev[0] > height_thresh)
            consecutives = consecutive(thresh_crossings[0])

            for each in consecutives:
                if peak in each:
                    ax[h].scatter(peak*frame_rate,test_peaks_lev[2][j])
                    ax[h].plot(each*frame_rate,np.repeat(height_thresh,len(each)))#
                    widths_lev.append(len(each)*frame_rate)
        
        
        
        test_peaks_dep = peaks_troughs(data.query('Muscle == "Depressor" and Filename == @g').value,data.query('Muscle == "Depressor" and Filename == @g').Time,smoothed = smoothed,peak_prom = peak_prom,threshold = threshold,title = g+" Depressor",graph=graphs)
        ax2[h].plot(np.arange(test_peaks_dep[0].shape[0])*frame_rate,test_peaks_dep[0])
        ax2[h].scatter(0,0,s=0)
        
        for j, peak in enumerate(test_peaks_dep[1]):
            height_thresh = test_peaks_dep[2][j]-cut_off
            thresh_crossings = np.where(test_peaks_dep[0] > height_thresh)
            consecutives = consecutive(thresh_crossings[0])

            for each in consecutives:
                if peak in each:
                    ax2[h].scatter(peak*frame_rate,test_peaks_dep[2][j])
                    ax2[h].plot(each*frame_rate,np.repeat(height_thresh,len(each)))#
                    widths_dep.append(len(each)*frame_rate)
        ax[h].set_ylim([0,105])
        ax2[h].set_ylim([0,105])
    fig.tight_layout()
    fig2.tight_layout()
    
    fig.savefig(condition+" levator.png")
    fig2.savefig(condition+" depressor.png")
    fig.savefig(condition+" levator.svg")
    fig2.savefig(condition+" depressor.svg")
    
    output = pd.DataFrame(widths_lev,columns = ["Widths"])
    output["Muscle"] = "Levator"
    output["Condition"] = condition
    
    ext = pd.DataFrame(widths_dep,columns = ["Widths"])
    ext["Muscle"] = "Depressor"
    ext["Condition"] = condition
    
    output = pd.concat([output,ext])
    
    return output

In [ ]:
all_widths2 = pd.DataFrame(columns = ["Widths","Muscle","Condition"])
for each in all_conditions.Condition.unique():
    widths = muscle_peak_width(all_conditions.query("Condition == @each and Time >= 0"),threshold,graphs='no',condition = each)
    all_widths2 = pd.concat([all_widths2,widths],ignore_index = True)

In [ ]:
#WIDTHS ONLY FOR POST STIM

widths_ATRstim = muscle_peak_width(ATR_wstim.query('Time >= 0'),threshold,graphs='no',condition = "ATR_wstim")
print(widths_ATRstim)

widths_ATRdecap = muscle_peak_width(ATR_decap.query('Time >= 0'),threshold,graphs='no',condition = "ATR_Decap")
print(widths_ATRdecap)

widths_ATRnostim = muscle_peak_width(ATR_nostim.query('Time >= 0'),threshold,graphs='no',condition = "ATR_nostim")
print(widths_ATRnostim)

widths_No_ATR_wstim = muscle_peak_width(NoATR_wstim.query('Time >= 0'),threshold,graphs='no',condition = "NoATR_wstim")
print(widths_No_ATR_wstim)

widths_No_ATRdecap = muscle_peak_width(NoATR_decap.query('Time >= 0'),threshold,graphs='no',condition = "NoATR_Decap")
print(widths_No_ATRdecap)

In [ ]:
all_widths = pd.concat([widths_ATRstim,widths_ATRdecap])
all_widths = pd.concat([all_widths,widths_ATRnostim])
all_widths = pd.concat([all_widths,widths_No_ATR_wstim])
all_widths = pd.concat([all_widths,widths_No_ATRdecap])

pval = spost.posthoc_dunn(all_widths,group_col="Condition",val_col = "Widths")

fig, ax = plt.subplots(figsize=[10,6])
sns.swarmplot(x="Condition",y="Widths",data=all_widths,color='k',size=2,order = ["NoATR_wstim","ATR_nostim","ATR_wstim","NoATR_Decap","ATR_Decap"])
sns.violinplot(x="Condition",y="Widths",data=all_widths,cut=0,palette = "GnBu_d",order = ["NoATR_wstim","ATR_nostim","ATR_wstim","NoATR_Decap","ATR_Decap"])
#plt.text(0.02, 0.03, "p = " + str(np.round(pval,8)), fontsize=12, transform=plt.gcf().transFigure)
#plt.savefig('Peak widths Plot.png')
plt.ylabel("Peak width in s",fontsize=15)
plt.savefig('Peak widths Plot.svg')


fig, ax = plt.subplots()
#sns.swarmplot(x="Condition",y="Widths",data=all_widths,color='k',size=3)
sns.boxplot(x="Condition",y="Widths",data=all_widths,palette = "GnBu_d")
plt.savefig('Peak widths Box.png')
print(pval)


In [ ]:
print(all_widths)